In [24]:
# tests to find solution for https://github.com/c-scale-community/use-case-hisea/issues/28

from datetime import timedelta, datetime
from pathlib import Path
import subprocess
import xarray as xr # note dependencies: dask, netCDF4

vars = ('thetao', 'bottomT', 'so', 'zos', 'uo', 'vo')
date_min = '2022-04-01'
date_max = '2022-04-05'

delta = datetime.strptime(date_max, '%Y-%m-%d') - datetime.strptime(date_min, '%Y-%m-%d')

var = vars[0]
i = 0
day = datetime.strptime(date_min, '%Y-%m-%d').date() + timedelta(days=i)
check_file = Path('/Users/backeb/Documents/data/tmp/cmems_'+str(var)+'_'+str(day)+'.nc')

j=0
while not check_file.is_file():
    print('j '+str(i))
    print(check_file.is_file())
    open(check_file, 'a')
    j+=1

print(check_file.is_file())
print(j)
    

j 0
False
True
1


In [32]:
#### IGNORE THIS, I WAS BEING A RETARD #### 
# works fine with xr.open_mfdataset('/Users/backeb/Documents/data/tmp/*no3*.nc')
# I was getting error in xr.open_mfdataset('/Users/backeb/Documents/data/tmp/*no3*.nc') BUT I WAS BEING AN IDIOT
# ValueError: Could not find any dimension coordinates to use to order the datasets for concatenation
# and using ds = xr.open_mfdataset('*no3*.nc', concat_dim='time', combine='nested') creates 25 time dimensions instead of the 5 expected

from pathlib import Path
import xarray as xr

datasets = []
#for path in Path('/Users/backeb/Documents/data/tmp/').iterdir():
for path in sorted(Path('/Users/backeb/Documents/data/tmp/').rglob('*no3*.nc')):
    print(path)
    ds = xr.open_dataset(path)
    datasets.append(ds)
combined = xr.concat(datasets, dim='time')

print(combined)

/Users/backeb/Documents/data/tmp/cmems_no3_2022-04-01.nc
/Users/backeb/Documents/data/tmp/cmems_no3_2022-04-02.nc
/Users/backeb/Documents/data/tmp/cmems_no3_2022-04-03.nc
/Users/backeb/Documents/data/tmp/cmems_no3_2022-04-04.nc
/Users/backeb/Documents/data/tmp/cmems_no3_2022-04-05.nc
<xarray.Dataset>
Dimensions:    (time: 25, depth: 50, latitude: 9, longitude: 9)
Coordinates:
  * depth      (depth) float32 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * latitude   (latitude) float32 36.5 36.75 37.0 37.25 ... 38.0 38.25 38.5
  * time       (time) datetime64[ns] 2022-04-01T12:00:00 ... 2022-04-05T12:00:00
  * longitude  (longitude) float32 22.5 22.75 23.0 23.25 ... 24.0 24.25 24.5
Data variables:
    no3        (time, depth, latitude, longitude) float32 0.2391 0.3665 ... nan
Attributes: (12/19)
    product:                       GLOBAL_ANALYSIS_FORECAST_BIO_001_028
    producer:                      CMEMS - Global Monitoring and Forecasting ...
    title:                         daily mean